<a href="https://colab.research.google.com/github/takadeanur/whisper/blob/main/Swallow_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentencepiece accelerate

In [ ]:
# @title Step.2 Tokenizer & Model Loading
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# @markdown [https://huggingface.co/tokyotech-llm](https://huggingface.co/tokyotech-llm) から利用したいモデルを選択してください。最初は 7b-instruct から始めるのがおすすめです。13bは ColabPro では動いています。70bはColabProでもダウンロードが難しいです。
tokenizer_model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']
model_name = "tokyotech-llm/Swallow-7b-instruct-hf" # @param ['tokyotech-llm/Swallow-7b-hf','tokyotech-llm/Swallow-7b-instruct-hf','tokyotech-llm/Swallow-13b-hf','tokyotech-llm/Swallow-13b-instruct-hf','tokyotech-llm/Swallow-70b-hf','tokyotech-llm/Swallow-70b-instruct-hf']

# Load
tokenizer = AutoTokenizer.from_pretrained(tokenizer_model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, low_cpu_mem_usage=True ,device_map="auto")

# @title
PROMPT_DICT = {
    "prompt_input": (
        "以下に、あるタスクを説明する指示があり、それに付随する入力が更なる文脈を提供しています。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 入力:\n{input}\n\n### 応答:"

    ),
    "prompt_no_input": (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{instruction}\n\n### 応答:"
    ),
}

def create_prompt(instruction, input=None):
    """
    Generates a prompt based on the given instruction and an optional input.
    If input is provided, it uses the 'prompt_input' template from PROMPT_DICT.
    If no input is provided, it uses the 'prompt_no_input' template.

    Args:
        instruction (str): The instruction describing the task.
        input (str, optional): Additional input providing context for the task. Default is None.

    Returns:
        str: The generated prompt.
    """
    if input:
        # Use the 'prompt_input' template when additional input is provided
        return PROMPT_DICT["prompt_input"].format(instruction=instruction, input=input)
    else:
        # Use the 'prompt_no_input' template when no additional input is provided
        return PROMPT_DICT["prompt_no_input"].format(instruction=instruction)

In [ ]:
# @title Step.3 Settings & Prompts
instruction_example = "以下のトピックにたいして少し面白い回答をしてください" # @param {type: "string"}
input_example = "日本の軍事力ランキングは" # @param {type: "string"}
Do_sample=True #@param {type:"boolean"}

if Do_sample:
  temperature = 0.99 #@param {type:"slider", min:0, max:2, step:0.1}
  top_p = 0.95 #@param {type:"slider", min:0, max:1, step:0.01}

max_new_tokens=128 #@param {type:"slider", min:128, max:1024, step:64}

# Example usage
# instruction_example = "以下のトピックに関する詳細な情報を提供してください。"
# input_example = "東京工業大学の主なキャンパスについて教えてください"
prompt = create_prompt(instruction_example, input_example)

input_ids = tokenizer.encode(
    prompt,
    add_special_tokens=False,
    return_tensors="pt"
)

tokens = model.generate(
    input_ids.to(device=model.device),
    max_new_tokens=max_new_tokens,
    temperature=temperature,
    top_p=top_p,
    do_sample=Do_sample,
)

out = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(out)